# Forecasting Product Sales

In this problem we aim to forecast sales of various products based on multiple time-series data. This solution was inspired by Mario Filho's tutorial on Multiple Time-Series Forecasting. This notebook was created using Google Colab.

In [0]:
# Importing packages
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression

In [0]:
# Mount Google Drive
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

In [0]:
# Saving the path and loading train DataFrame, filling empty as 0
path = '/content/gdrive/My Drive/Forecasting/'
data = pd.read_csv(path+'train.csv')
data = data.fillna(0)

In [0]:
# Creating a melt of the train DataFrame
melt = data.melt(id_vars='SKUID', var_name='Date', value_name='Qty')

In [0]:
# Date format change and sorting values in melt
import datetime
melt['Date'] = pd.to_datetime(melt['Date'], format="%d-%m-%Y")
melt = melt.sort_values(['Date', 'SKUID'])

In [0]:
# Encoding date using integer values - nth day
le = LabelEncoder()
melt['Date1'] = melt['Date']
melt = melt.drop(['Date'], axis = 1)
melt['Date'] = le.fit_transform(melt['Date1'])
melt = melt.drop(['Date1'], axis = 1)
melt

,SKUID,Qty,Date
0,1001,0.0,0
1,1003,0.0,0
2,1004,0.0,0
3,1005,0.0,0
4,1016,5.0,0
5,1022,0.0,0
6,1025,0.0,0
7,1028,0.0,0
8,1039,20.0,0
9,1042,133.0,0


In [0]:
# Copy of melt
melt1 = melt.copy()

In [0]:
# Defining the RMSLE function for error
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [0]:
# Creating features for previous 3 days' sales and difference in sales
melt1['Prev'] = melt1.groupby(['SKUID'])['Qty'].shift()
melt1['Diff'] = melt1.groupby(['SKUID'])['Qty'].diff()
melt1['Prev1'] = melt1.groupby(['SKUID'])['Qty'].shift(2)
melt1['Diff1'] = melt1.groupby(['SKUID'])['Prev1'].diff()
melt1['Prev2'] = melt1.groupby(['SKUID'])['Qty'].shift(3)
melt1['Diff2'] = melt1.groupby(['SKUID'])['Prev2'].diff()
melt1 = melt1.dropna()
melt1.head()

,SKUID,Qty,Date,Prev,Diff,Prev1,Diff1,Prev2,Diff2
6544,1001,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0
6545,1003,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0
6546,1004,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0
6547,1005,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0
6548,1016,13.0,4,14.0,-1.0,12.0,4.0,8.0,3.0


In [0]:
# Randome Forest Regressor - training
mean_error = []
for day in range(113,142):
    train = melt1[melt1['Date'] < day]
    val = melt1[melt1['Date'] == day]
    
    xtr, xts = train.drop(['Qty'], axis=1), val.drop(['Qty'], axis=1)
    ytr, yts = train['Qty'].values, val['Qty'].values
    
    mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
    mdl.fit(xtr, np.log1p(ytr))
    
    p = np.expm1(mdl.predict(xts))
    
    error = rmsle(yts, p)
    print('Day %d - Error %.5f' % (day, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

In [0]:
# Loading test DataFrame
testdf = pd.read_csv(path+'test.csv')

In [0]:
# Creating a DataFrame with train and test dates
total = pd.concat([data,testdf.drop('SKUID', axis = 1)], axis = 1)
total = total.fillna(0)

In [0]:
# Melt for all dates
meltt = total.melt(id_vars='SKUID', var_name='Date1', value_name='Qty')
meltt['Date1'] = pd.to_datetime(meltt['Date1'], format = "%d-%m-%Y")
meltt = meltt.sort_values(['Date1', 'SKUID'])

In [0]:
# Encoding the date - nth day
le = LabelEncoder()
meltt['Date'] = le.fit_transform(meltt['Date1'])
meltt = meltt.drop('Date1', axis = 1)
meltt1 = meltt.copy()

In [0]:
# Features created - previous sales and difference for last 9 days
meltt1['Prev'] = meltt1.groupby(['SKUID'])['Qty'].shift()
meltt1['Diff'] = meltt1.groupby(['SKUID'])['Qty'].diff()
meltt1['Prev1'] = meltt1.groupby(['SKUID'])['Qty'].shift(2)
meltt1['Diff1'] = meltt1.groupby(['SKUID'])['Prev1'].diff()
meltt1['Prev2'] = meltt1.groupby(['SKUID'])['Qty'].shift(3)
meltt1['Diff2'] = meltt1.groupby(['SKUID'])['Prev2'].diff()
meltt1['Prev3'] = meltt1.groupby(['SKUID'])['Qty'].shift(4)
meltt1['Diff3'] = meltt1.groupby(['SKUID'])['Prev3'].diff()
meltt1['Prev4'] = meltt1.groupby(['SKUID'])['Qty'].shift(5)
meltt1['Diff4'] = meltt1.groupby(['SKUID'])['Prev4'].diff()
meltt1['Prev5'] = meltt1.groupby(['SKUID'])['Qty'].shift(6)
meltt1['Diff5'] = meltt1.groupby(['SKUID'])['Prev5'].diff()
meltt1['Prev6'] = meltt1.groupby(['SKUID'])['Qty'].shift(7)
meltt1['Diff6'] = meltt1.groupby(['SKUID'])['Prev6'].diff()
meltt1['Prev7'] = meltt1.groupby(['SKUID'])['Qty'].shift(8)
meltt1['Diff7'] = meltt1.groupby(['SKUID'])['Prev7'].diff()
meltt1['Prev8'] = meltt1.groupby(['SKUID'])['Qty'].shift(9)
meltt1['Diff8'] = meltt1.groupby(['SKUID'])['Prev8'].diff()
meltt1 = meltt1.dropna()
meltt1

,SKUID,Qty,Date,Prev,Diff,Prev1,Diff1,Prev2,Diff2,Prev3,...,Prev4,Diff4,Prev5,Diff5,Prev6,Diff6,Prev7,Diff7,Prev8,Diff8
16360,1001,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16361,1003,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16362,1004,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16363,1005,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16364,1016,16.0,10,7.0,9.0,0.0,0.0,0.0,-5.0,5.0,...,13.0,0.0,13.0,-1.0,14.0,2.00,12.00,4.00,8.0,3.0
16365,1022,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16366,1025,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16367,1028,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0
16368,1039,25.0,10,10.0,15.0,20.0,0.0,20.0,10.0,10.0,...,0.0,-30.0,30.0,5.0,25.0,-20.00,45.00,2.00,43.0,23.0
16369,1042,379.0,10,302.0,77.0,150.0,38.0,112.0,-8.0,120.0,...,108.5,-39.5,148.0,-70.0,218.0,0.00,218.00,-31.00,249.0,116.0


In [0]:
# Extra Trees Regressor - training
l = []
for day in range(142,156):
    meltt1 = meltt.copy()
    meltt1['Prev'] = meltt1.groupby(['SKUID'])['Qty'].shift()
    meltt1['Diff'] = meltt1.groupby(['SKUID'])['Qty'].diff()
    meltt1['Prev1'] = meltt1.groupby(['SKUID'])['Qty'].shift(2)
    meltt1['Diff1'] = meltt1.groupby(['SKUID'])['Prev1'].diff()
    meltt1['Prev2'] = meltt1.groupby(['SKUID'])['Qty'].shift(3)
    meltt1['Diff2'] = meltt1.groupby(['SKUID'])['Prev2'].diff()
    meltt1['Prev3'] = meltt1.groupby(['SKUID'])['Qty'].shift(4)
    meltt1['Diff3'] = meltt1.groupby(['SKUID'])['Prev3'].diff()
    meltt1['Prev4'] = meltt1.groupby(['SKUID'])['Qty'].shift(5)
    meltt1['Diff4'] = meltt1.groupby(['SKUID'])['Prev4'].diff()
    meltt1['Prev5'] = meltt1.groupby(['SKUID'])['Qty'].shift(6)
    meltt1['Diff5'] = meltt1.groupby(['SKUID'])['Prev5'].diff()
    meltt1['Prev6'] = meltt1.groupby(['SKUID'])['Qty'].shift(7)
    meltt1['Diff6'] = meltt1.groupby(['SKUID'])['Prev6'].diff()
    meltt1['Prev7'] = meltt1.groupby(['SKUID'])['Qty'].shift(8)
    meltt1['Diff7'] = meltt1.groupby(['SKUID'])['Prev7'].diff()
    meltt1['Prev8'] = meltt1.groupby(['SKUID'])['Qty'].shift(9)
    meltt1['Diff8'] = meltt1.groupby(['SKUID'])['Prev8'].diff()
    meltt1 = meltt1.dropna()
    
    train = meltt1[meltt1['Date'] < day]
    val = meltt1[meltt1['Date'] == day]
    sku = np.asarray(val['SKUID'])
    
    xtr = train.drop(['Qty'], axis=1)
    ytr = train['Qty'].values
    xts = val.drop(['Qty'], axis=1)
    
    mdl = ExtraTreesRegressor(n_estimators=100)
    mdl.fit(xtr, ytr)
    
    p = mdl.predict(xts)
    p = np.around(p, decimals=2)
    l.append(p)
    print(p)
    d = dict()
    i = 0
    for s in sku:
      d[s] = p[i]
      i = i + 1
      meltt['Qty'] = np.where((meltt.Date==day)&(meltt.SKUID==s), d[s], meltt['Qty'])
print(l)

[0.   0.   4.97 ... 0.   0.   0.  ]
[0.   0.   1.82 ... 0.   0.   0.  ]
[0.   0.   0.35 ... 0.   0.   0.  ]
[0.   0.   0.19 ... 0.   0.   0.  ]
[0.   0.   0.12 ... 0.   0.   0.  ]
[0.   0.   0.09 ... 0.   0.   0.  ]
[0.   0.   0.03 ... 0.   0.   0.  ]
[0.   0.   0.16 ... 0.   0.   0.  ]
[0.   0.   0.04 ... 0.   0.   0.  ]
[0.   0.   0.02 ... 0.   0.   0.  ]
[0.   0.   0.01 ... 0.   0.   0.  ]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[array([0.  , 0.  , 4.97, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 1.82, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.35, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.19, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.12, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.09, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.03, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.16, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.04, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.02, ..., 0.  , 0.  , 0.  ]), array([0.  , 0.  , 0.01, ..., 0

In [0]:
# Creating the submission DataFrame and CSV file
df2 = meltt.pivot(index='SKUID', columns='Date')['Qty']
df2 = df2[[i for i in range(142, 156)]]
df2.columns = ['23-02-2019',	'24-02-2019',	'25-02-2019',	'26-02-2019',	'27-02-2019',	'28-02-2019',	'01-03-2019',	'02-03-2019',	'03-03-2019',	'04-03-2019',	'05-03-2019',	'06-03-2019',	'07-03-2019',	'08-03-2019']
df2.to_csv(path+'submit1.csv')
df2

,23-02-2019,24-02-2019,25-02-2019,26-02-2019,27-02-2019,28-02-2019,01-03-2019,02-03-2019,03-03-2019,04-03-2019,05-03-2019,06-03-2019,07-03-2019,08-03-2019
SKUID,,,,,,,,,,,,,,
1001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1003,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1004,4.97,1.82,0.35,0.19,0.12,0.09,0.03,0.16,0.04,0.02,0.01,0.00,0.00,0.00
1005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1016,1.77,0.96,0.72,0.26,0.06,0.03,0.15,0.06,0.03,0.01,0.00,0.00,0.00,0.00
1022,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1025,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1028,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1039,0.05,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Different regressors were implemented including Random Forest, Extra Trees and LGBM. A maximum accuracy score of about 87.6 was achieved using this method.